In [29]:
import importlib
import packages.word_corrector.src.core as core

importlib.reload(core)

<module 'packages.word_corrector.src.core' from '/home/jovyan/work/packages/word_corrector/src/core.py'>

In [1]:
!wget "https://docs.google.com/spreadsheets/d/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY/export?format=xlsx" -O data/test_corrections_with_expected.xlsx


--2025-05-25 07:12:44--  https://docs.google.com/spreadsheets/d/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 216.58.211.238, 2a00:1450:4026:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.211.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-14-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/97r5js7pudvi75h4k9pl3hkjpc/1748157165000/117963286932567830463/*/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY?format=xlsx [following]
--2025-05-25 07:12:46--  https://doc-14-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/97r5js7pudvi75h4k9pl3hkjpc/1748157165000/117963286932567830463/*/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY?format=xlsx
Resolving doc-14-1s-sheets.googleusercontent.com (doc-14-1s-sheets.googleusercontent.com)... 216.58.211.225, 2a00:1450:4026:808::2001
Connecting to doc-14-1s-sheets.googleusercontent.com

In [30]:
from packages.word_corrector.src.core import   PhraseCorrectorByWords, PhraseCorrectorNgrams, correct_text

In [3]:
import pandas as pd

In [4]:
df = pd.read_excel("data/test_corrections_with_expected.xlsx")

In [5]:
df[df["expected_correction"].notna()].head()

,file_name,text,corrected_text,original_text,expected_correction,comment
7,names/178.jpg,Sos-маска ARGANA\nOIL 270мл\n\nPROFESSIONAL дл...,SOS МАСКА ARGANA\nOIL 270МЛ\n\nPROFESSIONAL ДЛ...,Sos-маска ARGANA\nOIL 270мл\nPROFESSIONAL д/в...,SOS-МАСКА ARGANA\nOIL 270МЛ\nPROFESSIONAL ДЛЯ ...,Не удаляем дефисы и прочие знаки в первых двух...
9,names/877.jpg,Сахар белый\n500г\nкусовков кат. Экстра,САХАР БЕЛЫЙ\n500Г\nКУСОК КАТ ЭКСТРА,Сахар белый\n500г\nкусковой кат.Экстра,САХАР БЕЛЫЙ\n500Г\nКУСОВКОВ КАТ. ЭКСТРА,"Не применяем нормализацию к 3-ей строке, тольк..."
13,names/77.jpg,Продукция ТРИ ГОТОРА\n1.4л,ПРОДУКЦИЯ ТРИ ДОКТОРА\n1 4Л,Продукция ТРИ ТОПОРА\n1.4л\nплодовая алкогольн...,ПРОДУКЦИЯ ТРИ ДОКТОРА\n1.4Л,Не удаляем точки и спецсимволы на первых двух ...
15,names/896.jpg,Печенье ЧОКО БОЙ\n45г\nс обогащающей добавкой,ПЕЧЕНЬЕ ЧОКО БОЙ\n45Г\nС ОБОГАЩЕННЫЙ ДОБАВКА,Печенье ЧОКО БОЙ\n45г\nс обогащяющей добавкой,Печенье ЧОКО БОЙ\n45Г\nС ОБОГОЩАЮЩЕЙ ДОБАВКОЙ,"Не нормализуем третью строку, только upperscale"
17,names/159.jpg,прокладки OLA! SILK\nSENSE 60шт\nDAILY DEO РОМ...,ПРОКЛАДКИ OLA SILK\nSENSE 60ШТ\nDAILY DEO РОМА...,Прокладки OLA! SILK\nSENSE 60шт\nDAILY DEO РОМ...,ПРОКЛАДКИ OLA! SILK\nSENSE 60ШТ\nDAILY DEO РОМ...,Не нормализуем третью строку.\nНе удаляем спец...


In [6]:
df_new = df[df["expected_correction"].notna()].copy()

In [7]:
df_new["logs"] = None

# ByWords

In [33]:
text_corrector = PhraseCorrectorByWords(words_file="data/words.txt", score_cutoff=75,  min_len=2)
for index, row in df_new.iterrows():
    corrected_text, correct_logs = correct_text(row["text"],text_corrector)
    print("".join(["_"]*10))
    
    print("\nORIGINAL:\n", row['original_text'], "\n")
    print("\nLLM_TEXT:\n", row['text'], "\n")
    
    print("CORRECTED:\n", corrected_text, "\n")
    print("EXPECTED:\n", row['expected_correction'], "\n")
    print("COMMENTS:\n", row['comment'])
    print("\nLOGS_CORRECTIONS:\n", correct_logs)
    #df_new.at[index, "corrected_text"] = corrected_text
    #df_new.at[index, "logs"] = correct_logs

__________

ORIGINAL:
 Sos-маска ARGANA
OIL  270мл
PROFESSIONAL д/волос 


LLM_TEXT:
 Sos-маска ARGANA
OIL 270мл

PROFESSIONAL для волос 

CORRECTED:
 SOS-МАСКА ARGANA
OIL 270МЛ

PROFESSIONAL ДЛЯ ВОЛОС 

EXPECTED:
 SOS-МАСКА ARGANA
OIL 270МЛ
PROFESSIONAL ДЛЯ ВОЛОС 

COMMENTS:
 Не удаляем дефисы и прочие знаки в первых двух строках.
Удаляем пустые строки.
SOS-МАСКА я добавлю в базу одним словом.

LOGS_CORRECTIONS:
 []
__________

ORIGINAL:
 Сахар белый
500г
кусковой кат.Экстра 


LLM_TEXT:
 Сахар белый
500г
кусовков кат. Экстра 

CORRECTED:
 САХАР БЕЛЫЙ
500Г
КУСОВКОВ КАТ. ЭКСТРА 

EXPECTED:
 САХАР БЕЛЫЙ
500Г
КУСОВКОВ КАТ. ЭКСТРА 

COMMENTS:
 Не применяем нормализацию к 3-ей строке, только upperscale

LOGS_CORRECTIONS:
 []
__________

ORIGINAL:
 Продукция ТРИ ТОПОРА
1.4л
плодовая алкогольная полуслад. 


LLM_TEXT:
 Продукция ТРИ ГОТОРА
1.4л 

CORRECTED:
 ПРОДУКЦИЯ ТРИ ДОКТОРА
1.4Л 

EXPECTED:
 ПРОДУКЦИЯ ТРИ ДОКТОРА
1.4Л 

COMMENTS:
 Не удаляем точки и спецсимволы на первых двух строках



# ByNgrams

In [32]:
text_corrector = PhraseCorrectorNgrams(words_file="data/words.txt", score_cutoff=75,  min_len=2)
for index, row in df_new.iterrows():
    corrected_text, correct_logs = correct_text(row["text"],text_corrector)
    print("".join(["_"]*10))
    
    print("\nORIGINAL:\n", row['original_text'], "\n")
    print("\nLLM_TEXT:\n", row['text'], "\n")
    
    print("CORRECTED:\n", corrected_text, "\n")
    print("EXPECTED:\n", row['expected_correction'], "\n")
    print("COMMENTS:\n", row['comment'])
    print("\nLOGS_CORRECTIONS:\n", correct_logs)
    #df_new.at[index, "corrected_text"] = corrected_text
    #df_new.at[index, "logs"] = correct_logs

__________

ORIGINAL:
 Sos-маска ARGANA
OIL  270мл
PROFESSIONAL д/волос 


LLM_TEXT:
 Sos-маска ARGANA
OIL 270мл

PROFESSIONAL для волос 

CORRECTED:
 SOS-МАСКА ARGANA
OIL 270МЛ

PROFESSIONAL ДЛЯ ВОЛОС 

EXPECTED:
 SOS-МАСКА ARGANA
OIL 270МЛ
PROFESSIONAL ДЛЯ ВОЛОС 

COMMENTS:
 Не удаляем дефисы и прочие знаки в первых двух строках.
Удаляем пустые строки.
SOS-МАСКА я добавлю в базу одним словом.

LOGS_CORRECTIONS:
 [{'position': 0, 'original': 'SOS-МАСКА', 'corrected': 'SOS-МАСКА', 'score': 100.0}, {'position': 1, 'original': 'ARGANA', 'corrected': 'ARGANA', 'score': 100.0}]
__________

ORIGINAL:
 Сахар белый
500г
кусковой кат.Экстра 


LLM_TEXT:
 Сахар белый
500г
кусовков кат. Экстра 

CORRECTED:
 САХАР БЕЛЫЙ
500Г
КУСОВКОВ КАТ. ЭКСТРА 

EXPECTED:
 САХАР БЕЛЫЙ
500Г
КУСОВКОВ КАТ. ЭКСТРА 

COMMENTS:
 Не применяем нормализацию к 3-ей строке, только upperscale

LOGS_CORRECTIONS:
 [{'position': 0, 'original': 'САХАР', 'corrected': 'САХАР', 'score': 100.0}, {'position': 1, 'original': 'БЕЛЫЙ'

In [23]:
df_new[["text","corrected_text", "expected_text"]]

,file_name,text,corrected_text,original_text,expected_correction,comment,logs
7,names/178.jpg,Sos-маска ARGANA\nOIL 270мл\n\nPROFESSIONAL дл...,SOS-МАСКА ARGANA\nOIL 270МЛ\n\nPROFESSIONAL дл...,Sos-маска ARGANA\nOIL 270мл\nPROFESSIONAL д/в...,SOS-МАСКА ARGANA\nOIL 270МЛ\nPROFESSIONAL ДЛЯ ...,Не удаляем дефисы и прочие знаки в первых двух...,"[SOS-МАСКА ARGANA, OIL 270МЛ, , PROFESSIONAL д..."
9,names/877.jpg,Сахар белый\n500г\nкусовков кат. Экстра,САХАР БЕЛЫЙ\n500Г\nкусовков кат. Экстра,Сахар белый\n500г\nкусковой кат.Экстра,САХАР БЕЛЫЙ\n500Г\nКУСОВКОВ КАТ. ЭКСТРА,"Не применяем нормализацию к 3-ей строке, тольк...","[САХАР БЕЛЫЙ, 500Г, кусовков кат. Экстра]"
13,names/77.jpg,Продукция ТРИ ГОТОРА\n1.4л,ПРОДУКЦИЯ ТРИО ДОКТОРА\n1 4Л,Продукция ТРИ ТОПОРА\n1.4л\nплодовая алкогольн...,ПРОДУКЦИЯ ТРИ ДОКТОРА\n1.4Л,Не удаляем точки и спецсимволы на первых двух ...,"[ПРОДУКЦИЯ ТРИО ДОКТОРА, 1 4Л]"
15,names/896.jpg,Печенье ЧОКО БОЙ\n45г\nс обогащающей добавкой,ПЕЧЕНЬЕ ОКОН БЕЛОЙ\n45Г\nс обогащающей добавкой,Печенье ЧОКО БОЙ\n45г\nс обогащяющей добавкой,Печенье ЧОКО БОЙ\n45Г\nС ОБОГОЩАЮЩЕЙ ДОБАВКОЙ,"Не нормализуем третью строку, только upperscale","[ПЕЧЕНЬЕ ОКОН БЕЛОЙ, 45Г, с обогащающей добавкой]"
17,names/159.jpg,прокладки OLA! SILK\nSENSE 60шт\nDAILY DEO РОМ...,ПРОКЛАДКИ OLA! SILK\nSENSE 60ШТ\nDAILY DEO РОМ...,Прокладки OLA! SILK\nSENSE 60шт\nDAILY DEO РОМ...,ПРОКЛАДКИ OLA! SILK\nSENSE 60ШТ\nDAILY DEO РОМ...,Не нормализуем третью строку.\nНе удаляем спец...,"[ПРОКЛАДКИ OLA! SILK, SENSE 60ШТ, DAILY DEO РО..."


In [43]:
df_new.to_excel("tests_for_expected_correction.xlsx", index=False)